# Load Guild Tables

In [1]:
import os, sys
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import app_proj.excel_service as XS
print([x for x in dir(XS) if x[0]!='_' and len(x)>8])

['AppendToExcel', 'ReadSheet', 'ResetFile', 'ResetSheet']


In [6]:
DATA_PATH = os.path.join(MODULE_PATH, 'data')
print(DATA_PATH)

C:\Documents\Code\OpenStack\ThievesGuild\codebase\backend\emporium\data


In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Unlockable Thief

In [8]:
inputPath = os.path.join(DATA_PATH, 'GuildData.xlsx')
loadDf = XS.ReadSheet(inputPath, 'UnlockableThief')
loadDf = loadDf.dropna(subset=['Class'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

12

In [9]:
DB.DeleteTable('emporium', 'UnlockableThief')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'UnlockableThief', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'UnlockableThief')
print('records', len(checkLs))
PD.DataFrame(checkLs)[:4]

table deleted
bulk inserted
records 12


,id,Class,Stars,UnlockKeep,StoreCost,UnlockCost,StartTrait,RandomTraits
0,49,Burglar,1,0,2400,0,agi 3,0
1,50,Scoundrel,1,0,2400,0,cun 3,0
2,51,Ruffian,1,0,2400,0,mig 3,0
3,52,Burglar,2,1,3350,200,agi 4,2


### Unlockable Item

In [10]:
loadDf = XS.ReadSheet(inputPath, 'UnlockableItem')
loadDf = loadDf.dropna(subset=['Name'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

72

In [11]:
DB.DeleteTable('emporium', 'UnlockableItem')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'UnlockableItem', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'UnlockableItem')
print('records', len(checkLs))
PD.DataFrame(checkLs)[6:13]

table deleted
bulk inserted
records 72


,id,Name,Level,MagicLv,TotalLv,Slot,StoreCost,UnlockCost,Requirement,Trait,Combat,Skill,EnchantmentList
6,439,Cowl,1,0,1,head,800,NaN,None,None,None,per 5,None
7,440,Gloves,1,0,1,hands,800,NaN,None,None,None,sab 5,None
8,441,Boots,1,0,1,feet,800,NaN,None,None,None,tra 5,None
9,442,Helmet,1,0,1,head,800,NaN,None,None,def 1,None,None
10,443,Gauntlet,1,0,1,hands,800,NaN,None,None,dmg 1,None,None
11,444,Greaves,1,0,1,feet,800,NaN,None,None,att 1,None,None
12,445,Twin Daggers,1,1,2,weapon,1650,100.0,Burglar,agi 1,dmg 1,None,"dmg 1, def 1, per 5, tra 5"


### Thief Level

In [12]:
loadDf = XS.ReadSheet(inputPath, 'ThiefLevel')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf = loadDf.drop('RawPower', axis=1, errors='ignore')
loadDf.shape[0]

12

In [13]:
DB.DeleteTable('emporium', 'ThiefLevel')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'ThiefLevel', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'ThiefLevel')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 12


,id,Level,Experience,Power,TrainPeriod,WoundPeriod,KnockedOutPeriod
3,52,4,900,150,30 min,2 min,10 min
4,53,5,1280,210,1 hr,5 min,30 min
5,54,6,1700,275,2 hr,10 min,1 hr
6,55,7,2160,345,4 hr,30 min,2 hr


### Keep Upgrades

In [14]:
loadDf = XS.ReadSheet(inputPath, 'KeepUpgrades')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

9

In [15]:
DB.DeleteTable('emporium', 'KeepUpgrades')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'KeepUpgrades', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'KeepUpgrades')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 9


,id,Level,Wood,Stone,Iron,UpgradePeriod,MaxRoomCount,MaxRoomLevel,MaxThieves,GoldStorage,GemStorage,WoodStorage,StoneStorage,IronStorage
3,40,4,2730,1520,0,1 day,8,4,7,2150,170,1500,840,0
4,41,5,3350,1860,0,2 days,9,5,8,2480,200,1840,1030,0
5,42,6,4010,2220,0,3 days,10,6,8,2840,220,2210,1220,0
6,43,7,3140,1740,1050,5 days,10,7,9,3220,250,1730,960,580


### Singleton Upgrades

In [16]:
loadDf = XS.ReadSheet(inputPath, 'SingletonUpgrades')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

9

In [17]:
DB.DeleteTable('emporium', 'SingletonUpgrades')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'SingletonUpgrades', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'SingletonUpgrades')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 9


,id,Level,Wood,Stone,Iron,UpgradePeriod,KeepDefense
3,40,4,1820,1010,0,12 hr,trap level +1
4,41,5,2230,1240,0,1 day,defenders +1
5,42,6,2670,1480,0,2 days,trap level +1
6,43,7,2090,1160,700,3 days,defenders +1


### Storage Upgrades

In [18]:
loadDf = XS.ReadSheet(inputPath, 'StorageUpgrades')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

9

In [19]:
DB.DeleteTable('emporium', 'StorageUpgrades')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'StorageUpgrades', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'StorageUpgrades')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 9


,id,Level,Wood,Stone,Iron,UpgradePeriod,MaxThief,RecoveryBonus,GoldStorage,GemStorage,WoodStorage,StoneStorage,IronStorage,MaxThiefLevel
3,40,4,1820,1010,0,12 hr,3,5 min,2150,170,1500,840,0,7
4,41,5,2230,1240,0,1 day,4,5 min,2480,200,1840,1030,0,8
5,42,6,2670,1480,0,2 days,4,30 min,2840,220,2210,1220,0,9
6,43,7,2090,1160,700,3 days,5,30 min,3220,250,1730,960,580,10


### Factory Upgrades

In [20]:
loadDf = XS.ReadSheet(inputPath, 'FactoryUpgrades')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

9

In [21]:
DB.DeleteTable('emporium', 'FactoryUpgrades')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'FactoryUpgrades', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'FactoryUpgrades')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 9


,id,Level,Wood,Stone,Iron,UpgradePeriod,BonusGold,DailySlots,FenceStaff,BonusGems,UnlockSlots,JewelerStaff,BonusMaterials,WorkshopStaff,BonusXP,ReformatoryStaff
3,40,4,1820,1010,0,12 hr,+3% per 5 AGI,2,1,+45% per 5 CUN,2,1,+8% per 5 MIG,2,+3% per 3 END,2
4,41,5,2230,1240,0,1 day,+4% per 5 AGI,2,1,+45% per 5 CUN,3,1,+10% per 5 MIG,2,+4% per 3 END,2
5,42,6,2670,1480,0,2 days,+4% per 5 AGI,2,2,+45% per 5 CUN,3,2,+12% per 5 MIG,2,+5% per 3 END,2
6,43,7,2090,1160,700,3 days,+4% per 5 AGI,3,2,+60% per 5 CUN,3,2,+12% per 5 MIG,3,+5% per 3 END,3


In [22]:
infoLs = DB.GetTableCounts()
PD.DataFrame(infoLs)

,Module,Table,Count
0,members,User,2
1,emporium,UnlockableThief,12
2,emporium,UnlockableItem,72
3,emporium,ThiefLevel,12
4,emporium,KeepUpgrades,9
5,emporium,SingletonUpgrades,9
6,emporium,StorageUpgrades,9
7,emporium,FactoryUpgrades,9
8,emporium,Trap,72
9,emporium,Enemy,18
